# Step 2: 모델 학습 및 예측

## 📌 목적
Step 1에서 생성한 v4 데이터를 사용하여 2단계 계층적 분류 모델을 학습하고, 테스트 데이터를 예측합니다.

## 🔄 프로세스
1. **데이터 로드**: v4 train/test 데이터
2. **Stage 1 학습**: Rare(0,1) vs Others(2,3,4) 이진 분류 + Threshold 튜닝
3. **Stage 2A 학습**: Rare 내부 Segment 0 vs 1 분류
4. **Stage 2B 학습**: Others 내부 Segment 2 vs 3 vs 4 분류
5. **파이프라인 구성**: 전체 예측 함수 정의
6. **Test 예측**: 최종 제출 파일 생성

## 📊 출력물
- `models/model_stage1_rare_vs_others.pkl`: Stage1 모델
- `models/model_stage2A_seg01.pkl`: Stage2A 모델
- `models/model_stage2B_seg234.pkl`: Stage2B 모델
- `models/label_encoder_234.pkl`: Segment 2/3/4 인코더
- `results/v4_test_predictions.csv`: **최종 제출 파일**
- `results/v4_test_predictions.parquet`: 최종 제출 파일 (Parquet)

---

## 🛠️ 코드 실행

아래 셀을 실행하면 전체 파이프라인이 순차적으로 실행됩니다:
1. 데이터 로드 및 전처리
2. Stage 1, 2A, 2B 모델 학습
3. Validation 평가
4. Test 예측 및 제출 파일 생성

**예상 실행 시간**: 약 10-20분 (데이터 크기와 CPU 성능에 따라 다름)

---

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report,
    f1_score,
    precision_recall_curve,
    confusion_matrix
)
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

# =========================================
# 0) 경로 / 데이터 로드
# =========================================
PROJECT_ROOT = Path(r"C:\Users\User\전산통계프로젝트\final_submission")
DATA_DIR = PROJECT_ROOT / "data"
MODELS_DIR = PROJECT_ROOT / "models"
RESULTS_DIR = PROJECT_ROOT / "results"

# 원본 데이터 경로 (test ID 조회용)
ORIGINAL_DATA_DIR = Path(r"C:\Users\User\전산통계프로젝트\data")

print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATA_DIR   :", DATA_DIR)
print("MODELS_DIR :", MODELS_DIR)
print("RESULTS_DIR:", RESULTS_DIR)

train_path = DATA_DIR / "df_master_v4_train.parquet"
test_path  = DATA_DIR / "df_master_v4_test.parquet"

df_train = pd.read_parquet(train_path)
df_test  = pd.read_parquet(test_path)

print("\n[df_train shape]:", df_train.shape)
print("[df_test  shape]:", df_test.shape)
print("train columns 예시:", df_train.columns[:10].tolist())

# =========================================
# 1) rare_flag 생성 + FEATURE_COLS 정의
# =========================================
# Segment: 0/1 → rare, 2/3/4 → 일반
df_train["rare_flag"] = df_train["Segment"].isin([0, 1]).astype("int8")

FEATURE_COLS = [c for c in df_train.columns if c not in ["Segment", "rare_flag"]]

print("\n[FEATURE_COLS 개수]:", len(FEATURE_COLS))
print("FEATURE_COLS 예시:", FEATURE_COLS[:10])

# =========================================
# 2) Stage1: rare_flag (0/1) 이진 분류
# =========================================
X_stage1 = df_train[FEATURE_COLS]
y_stage1 = df_train["rare_flag"]

X_train, X_val, y_train, y_val = train_test_split(
    X_stage1,
    y_stage1,
    test_size=0.2,
    random_state=42,
    stratify=y_stage1
)

print("\n[Stage1 Train/Val]")
print("Train shape:", X_train.shape, " / rare 비율:", y_train.mean())
print("Val   shape:", X_val.shape, " / rare 비율:", y_val.mean())

scale_pos_weight = (1 - y_train.mean()) / y_train.mean()
print("\n[Stage1 scale_pos_weight]:", scale_pos_weight)

model_stage1 = XGBClassifier(
    objective="binary:logistic",
    max_depth=6,
    learning_rate=0.05,
    n_estimators=500,
    subsample=0.9,
    colsample_bytree=0.9,
    min_child_weight=1,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=-1,
)

model_stage1.fit(
    X_train,
    y_train,
    eval_set=[(X_val, y_val)],
    verbose=50
)

# --- threshold 튜닝 (F1 최댓값 기준) ---
val_prob = model_stage1.predict_proba(X_val)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_val, val_prob)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-9)

best_idx = np.argmax(f1_scores)
best_thr = thresholds[best_idx]

print("\n[Stage1 best threshold(F1 기준)]")
print("best_thr   :", best_thr)
print("Precision  :", precision[best_idx])
print("Recall     :", recall[best_idx])
print("F1         :", f1_scores[best_idx])

# confusion matrix
val_pred_bin = (val_prob >= best_thr).astype(int)
cm = confusion_matrix(y_val, val_pred_bin, labels=[0, 1])
tn, fp, fn, tp = cm.ravel()
print("\n[Stage1 Validation confusion_matrix] (row=true, col=pred)")
print(cm)
print("실제 rare 수(y=1):", y_val.sum())
print("rare로 예측된 수(pred=1):", val_pred_bin.sum())

recall_rare = tp / (tp + fn) if (tp + fn) > 0 else 0.0
precision_rare = tp / (tp + fp) if (tp + fp) > 0 else 0.0
print(f"→ Recall(rare): {recall_rare:.4f}")
print(f"→ Precision(rare): {precision_rare:.4f}")

# =========================================
# 3) Stage2A: Segment 0/1 분류 (희귀 내부)
# =========================================
stage2A_df = df_train[df_train["Segment"].isin([0, 1])].copy()
X_stage2A = stage2A_df[FEATURE_COLS]
y_stage2A = stage2A_df["Segment"]

print("\n[Stage2A 풀] (Segment 0/1)")
print("shape:", stage2A_df.shape)
print("Segment 분포:")
print(y_stage2A.value_counts().sort_index())

X_A_train, X_A_val, y_A_train, y_A_val = train_test_split(
    X_stage2A, y_stage2A,
    test_size=0.2,
    random_state=42,
    stratify=y_stage2A
)

print("\n[Stage2A Train/Val]")
print("Train shape:", X_A_train.shape)
print(y_A_train.value_counts().sort_index())
print("Val   shape:", X_A_val.shape)
print(y_A_val.value_counts().sort_index())

clf_A = XGBClassifier(
    objective="binary:logistic",
    max_depth=4,
    learning_rate=0.05,
    n_estimators=300,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42,
    n_jobs=-1,
)

clf_A.fit(
    X_A_train, y_A_train,
    eval_set=[(X_A_val, y_A_val)],
    verbose=50
)

y_A_pred = clf_A.predict(X_A_val)
print("\n[Stage2A Classification Report]")
print(classification_report(y_A_val, y_A_pred))
print("Stage2A Macro F1:", f1_score(y_A_val, y_A_pred, average="macro"))

# =========================================
# 4) Stage2B: Segment 2/3/4 분류 (일반 내부)
# =========================================
stage2B_df = df_train[df_train["Segment"].isin([2, 3, 4])].copy()
X_stage2B = stage2B_df[FEATURE_COLS]
y_stage2B = stage2B_df["Segment"]

print("\n[Stage2B 풀] (Segment 2/3/4)")
print("shape:", stage2B_df.shape)
print("Segment 분포:")
print(y_stage2B.value_counts().sort_index())

X_B_train, X_B_val, y_B_train, y_B_val = train_test_split(
    X_stage2B, y_stage2B,
    test_size=0.2,
    random_state=42,
    stratify=y_stage2B
)

print("\n[Stage2B Train/Val]")
print("Train shape:", X_B_train.shape)
print(y_B_train.value_counts().sort_index())
print("Val   shape:", X_B_val.shape)
print(y_B_val.value_counts().sort_index())

# 2/3/4 → 0/1/2 인코딩
le_234 = LabelEncoder()
y_B_train_enc = le_234.fit_transform(y_B_train)
y_B_val_enc   = le_234.transform(y_B_val)

# 클래스 가중치 (불균형 보정)
classes_B = np.unique(y_B_train_enc)
class_weights_B = {
    c: (len(y_B_train_enc) / (len(classes_B) * (y_B_train_enc == c).sum()))
    for c in classes_B
}
print("\n[Stage2B class_weights]")
for k, v in class_weights_B.items():
    print(f"  class {k} (원래 Segment {le_234.inverse_transform([k])[0]}): {v:.3f}")

sample_weight_B = pd.Series(y_B_train_enc).map(class_weights_B).values

clf_B = XGBClassifier(
    objective="multi:softprob",
    max_depth=7,
    learning_rate=0.05,
    n_estimators=700,
    subsample=0.9,
    colsample_bytree=0.9,
    min_child_weight=1,
    random_state=42,
    n_jobs=-1,
)

clf_B.fit(
    X_B_train, y_B_train_enc,
    sample_weight=sample_weight_B,
    eval_set=[(X_B_val, y_B_val_enc)],
    verbose=50
)

y_B_pred_enc = clf_B.predict(X_B_val)
y_B_pred = le_234.inverse_transform(y_B_pred_enc)

print("\n[Stage2B Classification Report (원래 Segment 기준)]")
print(classification_report(y_B_val, y_B_pred))
print("Stage2B Macro F1:", f1_score(y_B_val, y_B_pred, average="macro"))

# =========================================
# 5) 전체 파이프라인 함수 정의
# =========================================
def predict_v4_pipeline(X_input: pd.DataFrame) -> np.ndarray:
    """
    X_input: Segment/rare_flag 없는 feature DataFrame
             (컬럼 = FEATURE_COLS 그대로)
    """
    # 1) 컬럼 강제 정렬
    X = X_input[FEATURE_COLS].copy()

    n_samples = X.shape[0]
    y_final = np.full(n_samples, fill_value=-1, dtype=int)

    # -------------------------
    # 1단계: rare vs others
    # -------------------------
    prob_stage1 = model_stage1.predict_proba(X)[:, 1]
    y_pred_rareflag = (prob_stage1 >= best_thr).astype(int)

    rare_mask   = (y_pred_rareflag == 1)   # → 0/1
    others_mask = (y_pred_rareflag == 0)   # → 2/3/4

    # -------------------------
    # 2단계 A: rare 내부 (0 vs 1)
    # -------------------------
    if rare_mask.sum() > 0:
        X_rare = X.loc[rare_mask]
        y_rare_pred = clf_A.predict(X_rare)   # 0/1
        y_final[rare_mask] = y_rare_pred

    # -------------------------
    # 2단계 B: others 내부 (2/3/4)
    # -------------------------
    if others_mask.sum() > 0:
        X_others = X.loc[others_mask]
        y_others_pred_enc = clf_B.predict(X_others)           # 0/1/2
        y_others_pred = le_234.inverse_transform(y_others_pred_enc)  # 2/3/4
        y_final[others_mask] = y_others_pred

    if (y_final == -1).any():
        raise ValueError("y_final에 -1(미할당) 값이 있습니다. 마스크/인덱스 확인 필요.")

    return y_final

# =========================================
# 6) train 내부 Validation에서 v4 전체 평가
# =========================================
val_index = X_val.index  # Stage1에서 나눈 validation index

X_val_full = df_train.loc[val_index, FEATURE_COLS]
y_val_segment = df_train.loc[val_index, "Segment"]

print("\n[Val Segment 분포 (원래 0~4)]")
print(y_val_segment.value_counts().sort_index())

y_pred_v4 = predict_v4_pipeline(X_val_full)

print("\n[Stage1+Stage2A+Stage2B v4 최종 Classification Report (Val)]")
print(classification_report(y_val_segment, y_pred_v4))

macro_f1_v4 = f1_score(y_val_segment, y_pred_v4, average="macro")
print("Macro F1 (v4 전체 파이프라인, Val):", macro_f1_v4)

# =========================================
# 7) test 데이터에 최종 파이프라인 적용
# =========================================
print("\n=== v4 파이프라인을 test에 적용 ===")
print("[df_test shape]:", df_test.shape)

X_test_full = df_test[FEATURE_COLS]
y_test_pred = predict_v4_pipeline(X_test_full)   # 0~4

print("test 예측값 분포 (0~4):")
print(pd.Series(y_test_pred).value_counts().sort_index())

# 원래 레이블(A~E)로도 만들어두기
seg_map = {0: "A", 1: "B", 2: "C", 3: "D", 4: "E"}
y_test_pred_label = pd.Series(y_test_pred).map(seg_map)

# test ID 붙이기 (회원정보의 ID 사용)
test_member_path = DATA_DIR / "1.회원정보" / "test_회원정보.parquet"
df_test_member = pd.read_parquet(test_member_path)

print("\n[test_회원정보 shape]:", df_test_member.shape)

assert len(df_test_member) == len(df_test), "test 길이 안 맞음. v1 전처리 순서 확인 필요."

submission = pd.DataFrame({
    "ID": df_test_member["ID"].values,
    "Segment_pred": y_test_pred,
    "Segment_pred_label": y_test_pred_label.values
})

print("\n[submission head]")
print(submission.head())

out_parquet = DATA_DIR / "v4_test_predictions.parquet"
out_csv     = DATA_DIR / "v4_test_predictions.csv"

submission.to_parquet(out_parquet, index=False)
submission.to_csv(out_csv, index=False, encoding="utf-8-sig")

print("\n✅ Saved predictions:")
print(" -", out_parquet)
print(" -", out_csv)


PROJECT_ROOT: C:\Users\User\전산통계프로젝트
DATA_DIR   : C:\Users\User\전산통계프로젝트\data

[df_train shape]: (400000, 166)
[df_test  shape]: (100000, 165)
train columns 예시: ['Segment', '이용개월수_결제일_R6M', '평잔_일시불_6M', 'v4_limit_to_usage_ratio_R12M', '이용건수_체크_R12M', '이용금액_일시불_R3M', 'v4_balance_to_usage_ratio', '소지카드수_유효_신용', '잔액_신판ca최대한도소진율_r6m', '월중평잔_일시불']

[FEATURE_COLS 개수]: 165
FEATURE_COLS 예시: ['이용개월수_결제일_R6M', '평잔_일시불_6M', 'v4_limit_to_usage_ratio_R12M', '이용건수_체크_R12M', '이용금액_일시불_R3M', 'v4_balance_to_usage_ratio', '소지카드수_유효_신용', '잔액_신판ca최대한도소진율_r6m', '월중평잔_일시불', '한도증액후경과월']

[Stage1 Train/Val]
Train shape: (320000, 165)  / rare 비율: 0.000465625
Val   shape: (80000, 165)  / rare 비율: 0.0004625

[Stage1 scale_pos_weight]: 2146.6510067114095
[0]	validation_0-logloss:0.64567
[50]	validation_0-logloss:0.05095
[100]	validation_0-logloss:0.01170
[150]	validation_0-logloss:0.00619
[200]	validation_0-logloss:0.00405
[250]	validation_0-logloss:0.00297
[300]	validation_0-logloss:0.00252
[350]	validation_0-lo